In [ ]:
from google.cloud import bigquery
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

json = os.getenv("JSON_SUPER_SECRETO")

project = os.getenv("BQ_PROJECT")
dataset = os.getenv("BQ_DATASET")

#tabelas
tabela_items = os.getenv("BQ_TABELA_ITEMS")
tabela_discounts = os.getenv("BQ_TABELA_DISCOUNTS")
tabela_payments = os.getenv("BQ_TABELA_PAYMENTS")
tabela_torque = os.getenv("BQ_TABELA_TORQUE")
tabela_receipts = os.getenv("BQ_TABELA_RECEIPTS")


client = bigquery.Client.from_service_account_json(json)

In [203]:
#Query - Items
query_items = f"""
SELECT
    name,
    quantity,
    total_value,
    fk_receipt_identifier
FROM `{project}.{dataset}.{tabela_items}`
WHERE canceled = FALSE
ORDER BY name;
"""

query_itens_vendidos = client.query(query_items)
rows_itens_vendidos = query_itens_vendidos.result()
df_itens_vendidos = pd.DataFrame([dict(rows_itens_vendidos) for rows_itens_vendidos in rows_itens_vendidos])

In [ ]:
#Queries - Discounts
query_discounts = f"""
SELECT
    name,
    total,
    fk_receipt_identifier
FROM `{project}.{dataset}.{tabela_discounts}`
WHERE canceled = FALSE
ORDER BY name;
"""

query_descontos = client.query(query_discounts)
rows_descontos = query_descontos.result()
df_descontos = pd.DataFrame([dict(rows_descontos) for rows_descontos in rows_descontos])

In [ ]:
#Queries - Payments
query_payments = f"""
SELECT
    method,
    value,
    fk_receipt_identifier
FROM `{project}.{dataset}.{tabela_payments}`
ORDER BY method;
"""

query_pagamentos = client.query(query_payments)
rows_pagamentos = query_pagamentos.result()
df_pagamentos = pd.DataFrame([dict(rows_pagamentos) for rows_pagamentos in rows_pagamentos])

In [ ]:
#Queries - Torque
query_torque = f"""
SELECT
    shop_id,
    net_torque,
    net_torque_delivery,
    net_torque_in_store,
    operation_date
FROM `{project}.{dataset}.{tabela_torque}`
ORDER BY shop_id;
"""

query_torque_resultado = client.query(query_torque)
rows_torque_resultado = query_torque_resultado.result()
df_torque_resultado = pd.DataFrame([dict(rows_torque_resultado) for rows_torque_resultado in rows_torque_resultado])

In [ ]:
#Queries - Receipts
query_receipts = f"""
SELECT
    identifier,
    shop_id,
    total_value,
    delivery,
    date_time
FROM `{project}.{dataset}.{tabela_receipts}`
WHERE canceled = False AND staff = FALSE
ORDER BY date_time, shop_id;
"""

query_recibos = client.query(query_receipts)
rows_recibos = query_recibos.result()
df_recibos = pd.DataFrame([dict(rows_recibos) for rows_recibos in rows_recibos])

In [182]:
#agregar string unica
def unificar_string_unica(df):
    return ', '.join(df.unique())

In [183]:
#tratar a data da tabela receipts
df_recibos["operation_date"] = pd.to_datetime(df_recibos["date_time"]).dt.date

In [184]:
#Agregar consultas por fk

#itens por fk
df_items_agregar = df_itens_vendidos.groupby("fk_receipt_identifier").agg(
    nome_itens=("name", unificar_string_unica),
    qtd_itens=("quantity", "sum"),
    valor_itens=("total_value", "sum")
).reset_index().rename(columns={"fk_receipt_identifier": "identifier"})

#descontos por fk
df_descontos_agregar = df_descontos.groupby("fk_receipt_identifier").agg(
    nome_cupom_descontos=("name", unificar_string_unica),
    valor_descontos=("total", "sum")
).reset_index().rename(columns={"fk_receipt_identifier": "identifier"})

#pagamentos por fk
df_pagamentos_agregar = df_pagamentos.groupby("fk_receipt_identifier").agg(
    nome_metodo_pagamento=("method", unificar_string_unica),
    valor_pagamentos=("value", "sum")
).reset_index().rename(columns={"fk_receipt_identifier": "identifier"})

In [ ]:
#join por idenfificador - tabela Receipts como base e renomear para evitar KeyError
df_full = df_recibos.merge(df_items_agregar, on="identifier", how="left")
df_full = df_full.merge(df_descontos_agregar, on="identifier", how="left")
df_full = df_full.merge(df_pagamentos_agregar, on="identifier", how="left")

df_full = df_full.merge(
    df_torque_resultado[["shop_id", "operation_date", "net_torque", "net_torque_delivery", "net_torque_in_store"]],
    on=["shop_id", "operation_date"],#agrupar por data p/ evitar dados duplicados
    how="left"
)

df_full = df_full.drop(columns=["fk_receipt_identifier_x", "fk_receipt_identifier_y", "fk_receipt_identifier"], errors="ignore")

In [186]:
#tratar os nulos
df_full = df_full.fillna({
    "nome_itens": "",
    "qtd_itens": 0,
    "valor_itens": 0,
    "nome_cupom_descontos": "",
    "valor_descontos": 0,
    "nome_metodo_pagamento": "Não informado",
    "valor_pagamentos": 0,
    "delivery": False,
    "net_torque": 0,
    "net_torque_delivery": 0,
    "net_torque_in_store": 0
})

print("Qtd receipts:", df_recibos["identifier"].nunique())
print("Qtd linhas df_full:", df_full["identifier"].nunique())
print("Linhas totais df_full:", len(df_full))
print("Colunas do df_full:", df_full.columns.tolist())

Qtd receipts: 152897
Qtd linhas df_full: 152897
Linhas totais df_full: 152897
Colunas do df_full: ['identifier', 'shop_id', 'total_value', 'delivery', 'date_time', 'operation_date', 'nome_itens', 'qtd_itens', 'valor_itens', 'nome_cupom_descontos', 'valor_descontos', 'nome_metodo_pagamento', 'valor_pagamentos', 'net_torque', 'net_torque_delivery', 'net_torque_in_store']


In [187]:
list(df_full)
print(df_full["date_time"].dtype)

datetime64[ns, UTC]


In [188]:
#definir horário da emissão com base nos recibos
df_full["hora"] = df_full["date_time"].dt.hour
print(df_full["hora"])

#horas - período
def periodo(hora):
    if 6 <= hora < 12:
        return "Manhã"
    elif 12 <= hora < 18:
        return "Tarde"
    elif 18 <= hora < 24:
        return "Noite"
    else:#0h - 5h59
        return "Madrugada"
    
df_full["periodo"] = df_full["hora"].apply(periodo)

print(df_full["periodo"])

0         16
1         16
2         16
3         17
4         18
          ..
152892    23
152893    23
152894    23
152895    23
152896    23
Name: hora, Length: 152897, dtype: int32
0         Tarde
1         Tarde
2         Tarde
3         Tarde
4         Noite
          ...  
152892    Noite
152893    Noite
152894    Noite
152895    Noite
152896    Noite
Name: periodo, Length: 152897, dtype: object


In [189]:
#manter somente a data em date_time
df_full["date_time"] = df_full["date_time"].dt.date

df_full.rename(columns={"date_time": "data"}, inplace=True)

df_full = df_full.drop(columns=["operation_date", "hora"], errors="ignore")

print(df_full["data"])

0         2024-01-01
1         2024-01-01
2         2024-01-01
3         2024-01-01
4         2024-01-01
             ...    
152892    2025-06-30
152893    2025-06-30
152894    2025-06-30
152895    2025-06-30
152896    2025-06-30
Name: data, Length: 152897, dtype: object


In [191]:
#renomear nome lojas
df_full["shop_id"] = df_full["shop_id"].replace({
    "highway-praca-ekin": "Praça Ekin",
    "highway-avenida-nova-balanca": "Av. Nova Balança",
    "highway-rua-bens-perdidos": "R. Bens Perdidos",
    "highway-adidas-shopping": "Adidas Shopping"
})

In [193]:
list(df_full)
print(df_full.head(10))

                             identifier     shop_id  total_value  delivery  \
0  576edc4e-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         58.6      True   
1  576eda00-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         47.5      True   
2  576ed7bc-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         35.0     False   
3  576ed618-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         20.0     False   
4  576ed2e4-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         23.0     False   
5  576ec7ea-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         50.5      True   
6  576f1b6e-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         37.5     False   
7  576f1934-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         55.7      True   
8  576f15e2-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         62.4      True   
9  576f1088-e971-11ee-9aa3-0242ac1c000c  Praça Ekin         70.2      True   

         data                                         nome_itens  qtd_itens  \
0  2024-01-01        c/ frango teriyaki, sanduiches delivery 3

In [194]:
df_full.to_csv("visio_full.csv", index=False)